# Запуск Экспериментов в Azure ML

__Цель лабораторной работы:__

- научиться создавать и запускать Эксперименты (Experiments) Azure ML
- научиться получать детали выполнения Экспериментов.

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.datastore import Datastore

from datetime import datetime
import json

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Получим конфигурацию эксперимента:

In [ ]:
%run core.py

config = get_experiment_config('lab_4A')
config

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [ ]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

## Загрузка данных для Эксперимента

Просмотрим список зарегистрированных Наборов данных:

In [ ]:
print([ds for ds in Dataset.get_all(ws)])

Выберем `diabetes-data` в качестве Набора данных для Эксперимента:

In [ ]:
diabetes_ds = ws.datasets.get(config['core']['dataset_name'])

print(f'{diabetes_ds.name}: {diabetes_ds.description}')

Convert dataset to pandas dataframe and view:

In [ ]:
data = diabetes_ds.to_pandas_dataframe()
data.sample(10)

## Запуск Эксперимента

Создадим Эксперимент и залогируем основные данные об Эксперименте, а также информацию о данных используемых в нем:

In [ ]:
# Create an Azure ML experiment
experiment = Experiment(workspace=ws, name=config['experiment_name'])

# Create an interactive run
run = experiment.start_logging()

run.tag('healthcare')
run.tag('azure-ml-workshop')

run.log('Dataset dimension', f'{data.shape[0]}x{data.shape[1]}')
run.log('Workshop name', 'Winter school 2020')

print(f'Starting experiment {experiment.name}...')

## Анализ данных

Построим __гистограмму частот__, для того чтобы проанализировать количество пациентов с диабетом и без, а затем залогируем результаты:

In [ ]:
# Plot and log the count of diabetic vs non-diabetic patients
diabetic_counts = data['Diabetic'].value_counts()

fig = plt.figure(figsize=(6,6))
ax = fig.gca()    

diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name = 'label distribution', plot = fig)

# log distinct pregnancy counts
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

Построим __матрицу корреляции__ признаков в данных и залогируем результат:

In [ ]:
corr = data.corr()

ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

run.log_image(name='correlation matrix', plot=ax.get_figure())

Построим boxplot-графики:

In [ ]:
bxp = data.boxplot(column=['BMI', 'Age', 'PlasmaGlucose'], by='Diabetic', layout=(1, 3), figsize=(16, 9))

Посчитаем __описательные статистики__ датасета и залогируем результаты:

In [ ]:
# Log summary statistics for numeric columns
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat = keys[index], value = values[index])

Сохраним 100 случайных строк из данных в выходные данные Эксперимента:

In [ ]:
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream = './sample.csv')

## Завершение Эксперимента и просмотр результатов:

Завершим запуск Эксперимента, вызвав метод `complete()`:

In [ ]:
# Complete the run
run.complete()

Детали эксперимента:

In [ ]:
details = run.get_details()
print(details)

Метрики эксперимента:

In [ ]:
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

Выходные данные:

In [ ]:
files = run.get_file_names()
print(json.dumps(files, indent=2))

Визуализируем данные с помощью виджета:

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

Перейдите по сссылке, которая появилась в виджете (выше), чтобы просмотеть результаты Эксперимента на протале Azure ML.